![Multiple Chains](../assets/advanced-mutliple-chains.png)

---


### Learning objective:
By the end of this lesson, you will be able to practice chaining and multiple chaining.


### About:  
In this lesson students will supercharge their workflows by processing documents in a model and creating multiple chains. 


### Prerequisites:
- Python (required) 
- Intro to LangChain and prior prompt  eng. lessons (required) 
- Visual Studio Code (recommended)
- GitHub Copilot lessons (recommended) 

### Contents
1. [Imports](#imports)
1. [LCEL chains](#lcel)
1. [Multiple chains](#chains)

### Activities
1. [Lab](#lab)


<a id='imports'></a>
## Imports

In [1]:
#basic chains
from langchain_openai import ChatOpenAI #openai chatbot
from langchain_core.prompts import ChatPromptTemplate #template for chat prompts
from langchain_core.output_parsers import StrOutputParser #output parser for string output 

#documents 
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

<a id='lcel'></a>
## Chain Review 
**LCEL** allows you build complex chains with runnable components, "runnables" greatly reducing the amount of code you need to write for common model tasks. They come with common methods or interfaces that you can use in many situations. You quickly switch out components and create chains of chains (multiple chains). 

#### Basic Chain
- Language or chat model 
- Prompt Template
- Output parser


## LCEL Chains
In addition to custom chains you build, LangChain comes with a variety pre-built chains for common purposes you can use without having to specify each component. 

#### Example Pre-Built Chains
- [create_stuff_documents_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html#langchain.chains.combine_documents.stuff.create_stuff_documents_chain): "	This chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM. It passes ALL documents, so you should make sure it fits within the context window the LLM you are using."
- [load_query_constructor_runnable](https://api.python.langchain.com/en/latest/chains/langchain.chains.query_constructor.base.load_query_constructor_runnable.html#langchain.chains.query_constructor.base.load_query_constructor_runnable): "Can be used to generates queries. You must specify a list of allowed operations, and then will return a runnable that converts a natural language query into those allowed operations."
- [create_history_aware_retriever](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html#langchain.chains.history_aware_retriever.create_history_aware_retriever): "This chain takes in conversation history and then uses that to generate a search query which is passed to the underlying retriever."
- [and many more!](https://python.langchain.com/docs/modules/chains#lcel-chains)

#### Key Interfaces
- **Invoke (invoke):** Pass in a string and get back a string
- **Stream (stream):** Streams the response 
- **Batch (batch):** Pass in multiple prompts and get back the response 
- **Async (ainvoke):** Run asynchronously 

#### Docs
1. [LangChain LCEL](https://python.langchain.com/docs/expression_language/)
1. [LangChain Interface](https://python.langchain.com/docs/expression_language/interface)

In [2]:
# example from LangChain docs (link above for create_stuff_documents_chain)
prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)
llm = ChatOpenAI(openai_api_key="...", model_name="gpt-4")
chain = create_stuff_documents_chain(llm, prompt)

# sample documents 
docs = [
    Document(page_content="Sarah like blue but not green"),
    Document(page_content = "Juana loves yellow but not as much as she loves red")
]

chain.invoke({"context": docs})

"Peter is fond of green and doesn't care for red\n\nLiam prefers black over any other color\n\nEmma adores pink but can't stand brown\n\nOliver enjoys purple but dislikes orange\n\nSophia is partial to white and detests black\n\nNoah is a fan of orange but has a distaste for pink\n\nMia appreciates teal but does not like yellow\n\nLucas is a fan of brown but cannot stand blue\n\nHarper loves all colors but has a special affinity for rainbow\n\nElijah only likes the color gray and finds all other colors unappealing."

<a id='chains'></a>
## Multiple chains
Automate your work flows and build more complex systems by creating chains of chains. 

Generate a city and tourist destinations. 

![Prompt-Map](../assets/Additional%20Assets.png)


In [3]:
# Example 

model = ChatOpenAI(openai_api_key="...", model_name="gpt-4")

#chain 1 prompt and chain to output a city
prompt1 = ChatPromptTemplate.from_template("Name a popular city for tourists to visit in {month}.")
chain1 = prompt1 | model | StrOutputParser()

#chain 2 prompt and chain- uses city from prompt 1 to generate destinations 
prompt2 = ChatPromptTemplate.from_template(
    "What country is the {city} in? Name 3 popular tourist destinations in that country."
)
chain2 = (
    {"city": chain1} #include chain 1! 
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"month": "August"})

'Paris is in France. Three popular tourist destinations in France are the Eiffel Tower, Louvre Museum, and Palace of Versailles.'

<a id='trouble'></a>
## Troubleshooting
As we add more complexity to our chains, it will be important to see the mechanics behind LangChain to understand the results we are getting, to improve our prompts, and debug. 

- Review tools in LangChain (e.g., debug and verbose) to help track logs 
Demo each of debug and verbose 

#### Debug import and code: 
```python
from langchain.globals import set_debug

set_debug(True)
```

### Let's revist our prior example 

In [4]:
# Example chain setup 

model = ChatOpenAI(openai_api_key="...", model_name="gpt-4")

#chain 1 prompt and chain to output a city
prompt1 = ChatPromptTemplate.from_template("Name a popular city for tourists to visit in {month}.")
chain1 = prompt1 | model | StrOutputParser()

#chain 2 prompt and chain- uses city from prompt 1 to generate destinations 
prompt2 = ChatPromptTemplate.from_template(
    "What country is the {city} in? Name 3 popular tourist destinations in that country."
)
chain2 = (
    {"city": chain1} #include chain 1! 
    | prompt2
    | model
    | StrOutputParser()
)

### Let's invoke the chain with debug turned on

Read through each line. 
#### Take note of: 
- What city did the model generate in chain 1? 
- What was passed to chain 2? 


In [5]:
from langchain.globals import set_debug #import 
set_debug(True)
chain2.invoke({"month": "August"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "month": "August"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city>] Entering Chain run with input:
{
  "month": "August"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "month": "August"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city> > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "month": "August"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city> > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city> > 3:chain:RunnableSequence > 5:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Name a popular city for tourists to visit in August."
  ]
}
[llm/end] [1:chain:Run

'Paris is in the country of France. Three popular tourist destinations in France are the Eiffel Tower, Louvre Museum, and Palace of Versailles.'

In [6]:
# Great! Now let's turn off debug for now...
set_debug(False)


<a id='lab'></a>
## Lab

Build a series of 3 chains that 1) loads family preferences recommends a vacation destination, 2) it passes that vacation spot to a second model that recommends the best month to travel, 3) based on that month generates a 3-day travel itinerary. 

chain|chain type| input | output
--|--|--| --|
chain1 | create_stuff_documents_chain | family_docs | vacation destination 
chain2 | custom basic chain | vacation destination | month
chain3 | custom basic chain | month | 3-day itinerary

#### Family Travel Documents 

In [7]:
# Generated by GPT
family_docs = [
    Document(page_content="Sarah likes to travel to warm places but not cold ones"),
    Document(page_content="John prefers cities with a rich history but doesn't like crowded places"),
    Document(page_content="Emma loves beach vacations but is not a fan of hiking"),
    Document(page_content="Mike enjoys adventurous trips but doesn't like long flights from Wales")
]

#### Set Model

In [8]:
model = ChatOpenAI(openai_api_key="...", model_name="gpt-4")

### Chain 1
Use create_stuff_documents_chain to load family_docs and generate a vacation destination 

In [9]:
# add your code here!
prompt = ChatPromptTemplate.from_messages(
    [("system", "What city should this family travel:\n\n{context}.")]
)
llm = ChatOpenAI(openai_api_key="...", model_name="gpt-4")
chain1 = create_stuff_documents_chain(llm, prompt)

# build iteratively and test along the way 
# chain1.invoke({"context": family_docs})

### Chain 2
Build a new chain to pass that vacation destination to a model generate the best month to travel to this location

In [10]:
# add your code here!
#chain 1 prompt and chain to output a city
prompt2 = ChatPromptTemplate.from_template(
    "Recommend a month to travel to this {city}"
)
chain2 = (
    {"city": chain1} #include chain 1! 
    | prompt2
    | model
    | StrOutputParser()
)

# build iteratively and test along the way 
# chain2.invoke({"context": family_docs})

### Chain 3
Build a 3rd and final chain that generates a 3-day travel itinerary for the family 

In [11]:
# add your code here!
prompt3 = ChatPromptTemplate.from_template(
    "Recommend a 3-day travel itinerary with one destination each day in {city} during {month}"
)
chain3 = (
    {"city": chain2, "month": chain2} #include chain 1! 
    | prompt3
    | model
    | StrOutputParser()
)

# build iteratively and test along the way 
chain3.invoke({"context": family_docs})

"Day 1: Barcelona City Exploration\nStart your journey at the historic Gothic Quarter (Barri Gòtic), where you can admire the architectural masterpieces like the Barcelona Cathedral and the ancient city walls. From there, head towards La Rambla, a bustling street filled with markets, restaurants, and street performers. Don't forget to visit the famous La Boqueria market for mouth-watering local and international delicacies. In the afternoon, visit the iconic Sagrada Familia, a masterpiece of Antoni Gaudí. End your day with a panoramic view of the city from Park Güell, another Gaudí's creation.\n\nDay 2: Art and History Day\nStart your day at the Picasso Museum to see an extensive collection of Picasso's artworks. Then, visit the Museum of the History of Barcelona (MUHBA) to learn about the city's rich history. In the afternoon, head towards Montjuic Hill. Visit the Montjuic Castle, the National Art Museum of Catalonia (MNAC), and the Magic Fountain. The Magic Fountain's light and water

In [12]:
# With debug
set_debug(True)
chain3.invoke({"context": family_docs})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city,month>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city,month> > 3:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city,month> > 4:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city,month> > 4:chain:RunnableSequence > 5:chain:RunnableParallel<city>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city,month> > 3:chain:RunnableSequence > 4:chain:RunnableParallel<city>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<city,month> > 4:chain:RunnableSequence > 5:chain:RunnableParallel<city> > 6:ch

"Day 1: Explore Seville's Historical Centre\n\nStart your tour in the heart of Seville - the historical centre. Begin at the Seville Cathedral, the world's largest Gothic cathedral and third-largest church in the world. Don't miss climbing up the Giralda Tower for a stunning view of the city. Next, head to the Real Alcazar, a UNESCO World Heritage site, to marvel at the beautiful Moorish architecture and idyllic gardens. After lunch, stroll around the Barrio Santa Cruz, the old Jewish quarter, with its narrow streets, picturesque squares, and colorful houses. End the day at Plaza de España, a magnificent semi-circular plaza with a beautiful fountain in the center, where you can enjoy a leisurely boat ride in the canal.\n\nDay 2: Experience Semana Santa or Feria de Abril\n\nIf you're visiting in April, you're in for a treat as you'll get to witness either Semana Santa or Feria de Abril, depending on the dates. Semana Santa is a week-long religious festival featuring processions of float